In [ ]:
%matplotlib widget
import pathlib
rootdir = pathlib.Path(".").resolve()
import numpy as np
import abtem
from scipy.stats.qmc import PoissonDisk
from data_generation import generate_particle
import matplotlib.pyplot as plt

In [ ]:
atoms = []
size = 1500  # In Angstroms
rng = np.random.default_rng()
engine = PoissonDisk(
    d=2,
    radius=60,
    rng=rng,
    l_bounds=(0, 0),
    u_bounds=(size,) * 2,
)
sample = engine.fill_space()
for (dy, dx) in sample:
    at = generate_particle()
    at.translate([dy, dx, 0.])
    atoms.append(at)

In [ ]:
atoms_combined = atoms[0]
for a in atoms[1:]:
    atoms_combined.extend(a)

In [ ]:
xy_vac = 3
atoms_combined.center(xy_vac)
positions = atoms_combined.get_positions()
size = size + (xy_vac * 2)

In [ ]:
sampling = 0.25  # A / px
shape = (size / sampling,) * 2  #px
potential = abtem.Potential(
    atoms_combined,
    gpts=shape,
    slice_thickness=5,
    periodic=False,
    projection="infinite",
)

potential_array = potential.build(lazy=False)
array = potential_array.array.sum(axis=0)
m5, m95 = np.percentile(array.ravel(), (0.5, 99.5))
array -= m5
array /= (m95 - m5)
array = np.clip(array, 0., 1.)

In [ ]:
measurements = abtem.measurements.Images(
    array,
    sampling=sampling,
)
filtered_measurements = measurements.gaussian_filter(0.25)


In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    np.roll(filtered_measurements.array, (1000, 1000), axis=(0, 1)),
    cmap="gray"
)

In [ ]:
raise

In [ ]:
image = filtered_measurements.array
image = np.clip(image, 0.03, np.inf)
np.savez_compressed(
    rootdir / "particles.npz",
    data=image,
    extent=np.asarray((size / 10., size / 10.)),
)